In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
import httplib2 

from sklearn.datasets.base import Bunch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from pymongo import MongoClient


df = pd.read_csv('1_mil.csv')



cate = pd.read_csv('cate.csv')

list_lv1 = cate['lv1_eng'].unique().tolist()
cate['lv1_label'] = 0
cate['lv2_label'] = 0
u = 1
for lv1 in list_lv1:
    cate.loc[cate['lv1_eng']==lv1,['lv1_label']] = u
    list_lv2 = cate[cate['lv1_eng']==lv1]['lv2_eng'].unique().tolist()
    v = 1
    for lv2 in list_lv2:
        cate.loc[cate['lv2_eng']==lv2,['lv2_label']] = u*100 + v
        v += 1
    u += 1
    

cate['lv1_eng'] = cate['lv1_eng'].apply(lambda x: x.lower())
cate['lv2_eng'] = cate['lv2_eng'].apply(lambda x: x.lower())

df.head()

df.drop(columns=['level_2','lv3','Pic',''],inplace=True)

df.columns = ['keyword','lv1_1','lv1_2','lv1_3','lv2_1','lv2_2','lv2_3','lv2_4']

for col in df.columns.tolist():
    df[col] = df[col].apply(lambda x: x.lower() if x != None else None)
    df[col] = df[col].apply(lambda x: None if x == '' else x)

df.loc[(df['lv2_3'].isna()==True)&(df['lv2_4'].isna()==False),['lv2_3']] = df['lv2_4']
df.loc[(df['lv2_2'].isna()==True)&(df['lv2_3'].isna()==False),['lv2_2']] = df['lv2_3']
df.loc[(df['lv2_1'].isna()==True)&(df['lv2_2'].isna()==False),['lv2_1']] = df['lv2_2']


df.loc[((df['lv1_2'].isna()==True)|(df['lv1_2']=='other'))&(df['lv1_3'].isna()==False),['lv1_2']] = df['lv1_3']
df.loc[((df['lv1_1'].isna()==True)|(df['lv1_1']=='other'))&(df['lv1_2'].isna()==False),['lv1_1']] = df['lv1_2']

cate_dict = cate[['lv2_eng','lv2_label']].set_index(keys='lv2_label').to_dict(orient='dict')['lv2_eng']

# cate_dict = {}
# for i in range(len(cate)):
    

cate_dict_lv1_2_lv2 = {lv1:cate[cate['lv1_eng']==lv1]['lv2_eng'].tolist() for lv1 in cate['lv1_eng'].tolist()}

cate_dict_lv2_2_lv1 = {lv2:cate[cate['lv2_eng']==lv2]['lv1_eng'].tolist()[0] for lv2 in cate['lv2_eng'].tolist()}



df['lv2_label'] = df['lv2_1'].apply(lambda x: cate[cate['lv2_eng']==x]['lv2_label'].tolist()[0] if len(cate[cate['lv2_eng']==x]['lv2_label'].tolist()) > 0  else 0)

mil = pd.read_csv('1_mil.csv')

mil.dropna(inplace=True)

mil['aha_cate'] = mil['aha_cate'].apply(lambda x: x.lower())

cate_dict_rev = cate[['lv2_eng','lv2_label']].set_index(keys='lv2_eng').to_dict(orient='dict')['lv2_label']

mil['lv2_label'] = mil['aha_cate'].apply(lambda x: cate_dict_rev[x])

mil.head()

df.rename(columns = {'keyword':'name'},inplace=True)

df_data = pd.concat([mil[['name','lv2_label']],df[['name','lv2_label']]])

df_data.reset_index(drop=True,inplace=True)

items = []
for i in range(len(df_data)):
    items += [{"keyword":df_data.loc[i]['name'],"label":df_data.loc[i]['lv2_label']}]

# a = df['lv2_1'].unique().tolist()

# b = cate['lv2_eng'].unique().tolist()

# b

# for i in a:
#     if i not in b:
#         print(i)

# [x.lower() for x in df.columns.tolist()]

# Get training data
data = []
target = []
for item in items:
    data.append(item["keyword"].lower())
    target.append(item["label"])
train = Bunch(data=data, target=target)

# Build Text classification model
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)),
                     ])
text_clf = text_clf.fit(train.data, train.target)



# import gdown
# file_id = '1--yNhXLC3_YHg8ctMiFN_lu1NzakkmZD'
# output = '1_mil.csv' #name of output file after download
# url = 'https://drive.google.com/uc?id='+file_id
# gdown.download(url ,output,quiet=False)



In [ ]:
a = input()
cate_dict.get(text_clf.predict([a])[0])